# Root area ratio



## Initialise code

First, we need to load some python packages and functions. This will enable us to reading data from csv files, define units, and plot any results.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyrootmemo.io import read_csv_roots
from pyrootmemo.helpers import units

## Load data

There are two sources of data available in the "sbee2025/data" folder:

* root intersection data for *Lolium perenne* (Talveg dataset)
* root intersection for *Onobrychis viciifolia* (Talveg dataset)

By default, this notebook loads the *Lolium perenne* data but you can easily change this out by changing the path to the relevant data file.

In [ ]:
# load tensile strength data from csv
data = read_csv_roots('data/experiment_lolium_perenne_talveg_20170615.csv')
# data = read_csv_roots('data/experiment_onobrychis_viciifolia_talveg_20170615.csv')

Your data is loaded into the object `data`. This object has attributes:

*  `diameter`: the diameter of each individual root
* `x0`: x-position of each root intersection with the window
* `y0`: y-position (depth) of each root intersection with the window

These data keeps its measurement units with it, so there is never any confusion as to what it units are! 

We can access these attributes as follows:


In [ ]:
# show data to screen using the 'print' command
print(data.diameter)

In [ ]:
# show only values (with no units, i.e. the "magnitude" of the measurement)
print(data.diameter.magnitude)

In [ ]:
# show the unit of measurement
print(data.diameter.units)

We can easily convert between different unit types by using the `.to()` method.

In [ ]:
# show diameter values in inches
print(data.diameter.to('inch'))

## Plotting

### Histogram of all diameters

Let's plot a histogram showing the distribution of measured diameters.

In [ ]:
# plot a histogram
plt.hist(data.diameter.magnitude)

# set axis labels
plt.xlabel('Diameter [' + str(data.diameter.units) + ']')
plt.ylabel('Number of roots [-]')

## Total root area ratio

In order to calculate any root area ratios we need to define the size of the window.

First, manually set the x (horizontal) and y (depth) position of the sides of the window. 

In [ ]:
window_x = [0, 26] * units('cm')
window_y = [0, 20] * units('cm')

We can now calculate the area of the window:

In [ ]:
window_width = window_x[1] - window_x[0]
window_height = window_y[1] - window_y[0]
window_area = window_width * window_height
print(window_area)

The total root area ratio is defined as the total cross-sectional area of the root divided by the cross-sectional area of the soil. Let's do some calculations:

In [ ]:
root_area_perroot = np.pi * (data.diameter / 2)**2
root_area_ratio_perroot = root_area_perroot / window_area
root_area_ratio = sum(root_area_ratio_perroot)
print(root_area_ratio)

Root diameters and window dimensions may have different units (e.g. millimetres 
and centimetres). The root area ratio is now defined with unusual units. 
Therefore, let us convert the calculated root area ratio to a more 
user-friendly unit, such as a dimensionless unit (e.g. mm^2/mm^2) or 
a percentage:

In [ ]:
print(root_area_ratio.to("mm^2/mm^2"))
print(root_area_ratio.to("%"))

## Locations of each individual root

Let's also have a look at the locations of each root, using a scatter plot.

In [ ]:
# set measurement unit for axes
axes_units = 'mm'

# plot a scatter plot
fig, ax = plt.subplots()
scatter = plt.scatter(
    data.x0.to(axes_units).magnitude,  # x-position
    data.y0.to(axes_units).magnitude,  # y-position
    c = data.diameter.magnitude,       # colours 
    alpha = 0.7                        # transparancy of marker points
    )

# set the axes
ax.set_aspect('equal', 'box')
ax.set_xlabel('x0 [' + axes_units + ']')
ax.set_ylabel('y0 [' + axes_units + ']')
ax.set_xlim(window_x.to(axes_units).magnitude)
ax.set_ylim(reversed(window_y.to(axes_units).magnitude))

# add a colour bar
fig.colorbar(
    scatter, 
    ax = ax, 
    label = 'Root diameter [' + str(data.diameter.units) + ']'
    )

## Root area ratio per grid cell and diameter range

Now make a plot that shows the root area ratio in each grid cell. 

First, define the number of individual bins in the 'x' and 'y' directions.

With these defined, we can calculte the x and y positions of the edges of 
each bin.

In [ ]:
# set the number of depth intervals (y) and columns (x)
x_bins = 5
y_bins = 4

# calculate the positions of the edges of all bins
x_edges = np.linspace(window_x[0], window_x[1], x_bins + 1)
y_edges = np.linspace(window_y[0], window_y[1], y_bins + 1)

We could also limit our plot to a certain diameter range, defined by a minimum 
and a maximum root diameter. With these, we can create a data 'mask' that helps 
us select only those roots that are within the specified diameter range.

In [ ]:
# set the minimum and maximum diameter of all roots to include in the plot
diameter_min = 0 * units('mm')
diameter_max = 100 * units('mm')

# create a data mask
mask = (data.diameter >= diameter_min) & (data.diameter <= diameter_max)

Now calculate the root area ratio contribution of each root to the window grid cell
it is part of:

In [ ]:
# area of each grid cell
grid_area = window_area / (x_bins * y_bins)
# contribution of each individual root to the root area ratio in each grid cell
rar_grid_contribution = root_area_perroot / grid_area


With these values, we can plot a 2-dimensional histogram

In [ ]:
# unit for root area ratio
axes_unit = 'mm'
rar_unit = '%'

# plot 2-dimensional histogram
fig, ax = plt.subplots()
handles = ax.hist2d(
    data.x0[mask].to(axes_units).magnitude,  # x-positions of each root
    data.y0[mask].to(axes_units).magnitude,  # y-positions of each root
    bins = [
        x_edges.to(axes_units).magnitude, 
        y_edges.to(axes_units).magnitude
        ],                                   # bins settings
    weights = rar_grid_contribution.to(rar_unit).magnitude,
    cmap = "Blues"
)

# set the axes
ax.set_aspect('equal', 'box')
ax.set_xlabel('x0 [' + axes_units + ']')
ax.set_ylabel('y0 [' + axes_units + ']')
ax.set_xlim(window_x.to(axes_units).magnitude)
ax.set_ylim(reversed(window_y.to(axes_units).magnitude))

# add a colour bar for the root area ratio
fig.colorbar(
    handles[3], 
    ax = ax, 
    label = 'Root area ratio [' + rar_unit + ']'
    )

## Explore!

You can now investigate

* How does the root area ratio change with depth? You could look at this by
  changing the number of y bins, and setting the number of bins in the 
  x-direction to 1.

* What is the contribution of different diameter ranges to the total root 
  area ratio?. Does most of the root area ratio come from relatively thin 
  or relatively thick roots?

* Vary the number of bins in x and y-directions. What do you think is a 
  reasonable bin size for this data?
  
  * ...too many bins and we get a lot of scatter...
  * ...too few bins and we lose too much information about the location of each root...

* Now use a different dataset, for example the data for
  *Onobrychis viciifolia*. Are your results and choices different?